In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

In [ ]:
from tensorflow.keras.applications import InceptionV3

In [ ]:
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#print("num GPUs available: ", len(physical_devices))
#tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
pre_model = InceptionV3(
    include_top = False,
    weights = None,
    input_shape = (224, 224, 3)
)

In [ ]:
pre_model.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
for layer in pre_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_model.get_layer('mixed8')

In [ ]:
x = Flatten()(last_layer.output)
#x = Dense(512, activation = 'relu')(x)
#x = Dropout(0.2)(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.1)(x)
x = Dense(1, activation = 'sigmoid')(x)

In [ ]:
model = Model(pre_model.input, x)

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_generator = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    horizontal_flip = True,
    width_shift_range = 0.1,
    shear_range = 15,
    zoom_range=0.1,
    height_shift_range = 0.3,
    channel_shift_range = 10
).flow_from_directory(
    'Satellite Info/train',
    target_size = (224, 224),
    class_mode = 'binary',
    batch_size = 50
)

validation_generator = ImageDataGenerator(
    rescale = 1./255.
).flow_from_directory(
    'Satellite Info/validation',
    target_size = (224, 224),
    class_mode = 'binary',
    batch_size = 50
)

test_generator=ImageDataGenerator(
rescale = 1./255.
).flow_from_directory('Satellite Info/test',
    target_size=(224,224),
    class_mode='binary',
    batch_size = 15
                    )

In [ ]:
model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 30
)